In [33]:
import xml.etree.ElementTree as ET
import re
from nltk.corpus import stopwords
import pandas as pd

In [34]:
# read the latest mass production table
# for 6.05 clean
# infile_table = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
# infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# outfile_folder = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 6.10 clean
infile_brd = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/MassProduction/finalTemplateNew_cleaned.brd"
infile_table = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/MassProduction/6.10_table_new_cleaned.txt"
outfile_folder = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/FinalBRDs/"

# for 7.17 clean
# infile_table = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
# infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# outfile_folder = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

# for 6.05 translate
# infile_table = "./Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt"
# infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# outfile_folder = "./Output_translated_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 translate
# infile_table = "./Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt"
# infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# outfile_folder = "./Output_translated_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

In [35]:
table_new = pd.read_csv(infile_table, sep="\t", index_col=0, keep_default_na=False)

In [36]:
# replace variable with value in the latest mass production table
# iterate column and row in table_new
for column in range(table_new.shape[1]):
    for row in range(table_new.shape[0]):
        content_new = str(table_new.iloc[row, column])
        pattern_variable = "%\((.*?)\)%"
        # count the number of the replacement in one variable(content_new)
        match_variable = [match for match in re.findall(pattern_variable, str(content_new))]
        for i in range(len(match_variable)):
            # find the variable
            variable = "%(" + str(match_variable[i]) + ")%"
            # find the corresponding column name, and then find the value
            column_name = table_new.columns[column]
            try:
                value = table_new.loc[variable, column_name] 
                # print(variable, value)
                content_new = content_new.replace(variable, value)
                table_new.iloc[row, column] = content_new
            except:
                print(column_name, variable + " doesn't exist")

1 %(hint_r2_left_level_1)% doesn't exist
1 %(hint_r2_left_level_2)% doesn't exist
1 %(hint_r2_left_level_3)% doesn't exist
1 %(hint_r2_center_level_1)% doesn't exist
1 %(hint_r2_center_level_2)% doesn't exist
1 %(hint_r2_center_level_3)% doesn't exist
1 %(hint_r2_right_level_1)% doesn't exist
1 %(hint_r2_right_level_2)% doesn't exist
1 %(hint_r2_right_level_3)% doesn't exist
1 %(hint_r3_left_level_1)% doesn't exist
1 %(hint_r3_left_level_2)% doesn't exist
1 %(hint_r3_left_level_3)% doesn't exist
1 %(hint_r3_center_level_1)% doesn't exist
1 %(hint_r3_center_level_2)% doesn't exist
1 %(hint_r3_center_level_3)% doesn't exist
1 %(hint_r3_right_level_1)% doesn't exist
1 %(hint_r3_right_level_2)% doesn't exist
1 %(hint_r3_right_level_3)% doesn't exist
1 %(hint_done)% doesn't exist
1 %(hint_buggy)% doesn't exist
1 %(noTradeValue_r2_left)% doesn't exist
1 %(noTradeValue_r2_center)% doesn't exist
1 %(noTradeValue_r2_right)% doesn't exist
1 %(noTradeValue_r3_left)% doesn't exist
1 %(noTradeValue

In [37]:
# table_new.head()
for i in range(len(table_new.columns)):
    column_name = table_new.columns[i]
    print(table_new.columns[i])
    fout = outfile_folder + str(table_new.columns[i]) + ".brd"
    # print(fout)
    count_line = 0
    count_text = 0
    count_name = 0
    with open(infile_brd, 'r') as infile, open(fout, 'w+') as outfile:
        for line in infile:
            line = line.replace('\r', '')
            line_str = str(line)
            # replace massproduce
            # print(line_str)
            if count_name == 0:
                pattern_problem_name_1 = "<ProblemName>(.*?)</ProblemName>"
                match_problem_name_1 = [match for match in re.findall(pattern_problem_name_1, str(line_str))]
                pattern_problem_name_2 = "<ProblemName />"
                match_problem_name_2 = [match for match in re.findall(pattern_problem_name_2, str(line_str))]
                problem_name = "<ProblemName>" + str(column_name) + "</ProblemName>"
                if match_problem_name_1:
                    line_str = line_str.replace(match_problem_name_1[0], problem_name)
                    print(line_str)
                    count_name += 1
                elif match_problem_name_2:
                    line_str = line_str.replace(match_problem_name_2[0], problem_name)
                    print(line_str)
                    count_name += 1
                else:
                    pass
            # replace text in first node
            if count_text == 0:
                start_pattern_first_node = '<text>'
                start_index_first_node = [match.start() for match in re.finditer(start_pattern_first_node, line_str)]
                end_pattern_first_node = '</text>'
                end_index_first_node = [match.start() for match in re.finditer(end_pattern_first_node, line_str)]
                if start_index_first_node != []:
                    count_text += 1
                    # print(line_str[start_index_first_node[0]+6: end_index_first_node[0]])
                    text_old = line_str[start_index_first_node[0]+6: end_index_first_node[0]]
                    line_str = line_str.replace(text_old, column_name)
            # count the number of the replacement in one variable(line_str)
            pattern_variable = "%\((.*?)\)%"
            # count the number of the replacement in one variable(content_new)
            match_variable = [match for match in re.findall(pattern_variable, str(line_str))]
            if match_variable == []:
                line_str = line_str
            else:
                for j in range(len(match_variable)):
                    # find the variable
                    variable = "%(" + str(match_variable[j]) + ")%"
                    # find the corresponding column name, and then find the value
                    try:
                        value = table_new.loc[variable, column_name] 
                        # print(variable, value)
                        line_str = line_str.replace(variable, value).replace("<%", "&lt;%").replace("%>", "%&gt;")
                    except:
                        print(column_name, variable + " doesn't exist") 
            count_line += 1    
            outfile.write(line_str)
        print(fout.split("/")[-1] + " finished")

1
                <ProblemName>1</ProblemName>

1 %(r1c2_min)% doesn't exist
1 %(r1c2_max)% doesn't exist
1 %(link_r2_left_min)% doesn't exist
1 %(link_r2_left_max)% doesn't exist
1 %(link_r2_left)% doesn't exist
1 %(link_r2_left)% doesn't exist
1 %(r2_left_column)% doesn't exist
1 %(r2_left_column)% doesn't exist
1 %(link_r2_left_min)% doesn't exist
1 %(link_r2_left_max)% doesn't exist
1 %(link_r2_left)% doesn't exist
1 %(link_r2_left)% doesn't exist
1 %(r2_left_column)% doesn't exist
1 %(r2_left_column)% doesn't exist
1 %(link_r2_center_min)% doesn't exist
1 %(link_r2_center_max)% doesn't exist
1 %(link_r2_center)% doesn't exist
1 %(link_r2_center)% doesn't exist
1 %(r2_center_column)% doesn't exist
1 %(r2_center_column)% doesn't exist
1 %(link_r2_center_min)% doesn't exist
1 %(link_r2_center_max)% doesn't exist
1 %(link_r2_center)% doesn't exist
1 %(link_r2_center)% doesn't exist
1 %(r2_center_column)% doesn't exist
1 %(r2_center_column)% doesn't exist
1 %(link_r2_right_min)% doesn'